In [5]:
import pandas as pd
import feather
from datetime import datetime

In [3]:
df = feather.read_dataframe(r"C:\Users\Scott\Documents\data\cat_dashboard\BATT.feather")

# Change Format

In [6]:
unique_period = df.PERIODCODE.unique()
period_map = {k:datetime.strptime(str(v),'%Y14%m') for k,v in zip(unique_period, unique_period)}
df['PERIODCODE'] = df['PERIODCODE'].map(period_map)

# Lag1 Plot

In [30]:
use_cols = ['PERIODCODE', 'PVAL']
lag1_df = (
    df[use_cols]
    .assign(Lag1 = df.PVAL.shift(1))
    .set_index('PERIODCODE').resample('1d').sum()
)

In [31]:
zero_rows = (lag1_df.PVAL == 0) & (lag1_df.Lag1 == 0)
lag1_df = lag1_df[~zero_rows]

In [ ]:
fig, ax = plt.subplots()
ax = sns.regplot(x='Lag1', y='PVAL', data=lag1_df, ci=None, scatter_kws=dict(color='0.25'))
ax.set_aspect('equal')
ax.set_title('Lag Plot of BATT Sales')

# pandas tips
## pd.df.align

> Also support series.

In [40]:
df = pd.DataFrame(
    [[1, 2, 3, 4], [6, 7, 8, 9]], columns=["D", "B", "E", "A"], index=[1, 2]
)
other = pd.DataFrame(
    [[10, 20, 30, 40], [60, 70, 80, 90], [600, 700, 800, 900]],
    columns=["A", "B", "C", "D"],
    index=[2, 3, 4],
)

In [43]:
from IPython.core.display_functions import display

display(df)
display(other)

,D,B,E,A
1,1,2,3,4
2,6,7,8,9


,A,B,C,D
2,10,20,30,40
3,60,70,80,90
4,600,700,800,900


### Outer Join, Axis 0

In [53]:
left, right = df.align(other, join="outer", axis=0)
print("Left")
display(left)
print("Right")
display(right)

Left


,D,B,E,A
1,1.0,2.0,3.0,4.0
2,6.0,7.0,8.0,9.0
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


Right


,A,B,C,D
1,NaN,NaN,NaN,NaN
2,10.0,20.0,30.0,40.0
3,60.0,70.0,80.0,90.0
4,600.0,700.0,800.0,900.0


### Outer Join, Axis 1

In [48]:
left, right = df.align(other, join="outer", axis=1)
print("Left")
display(left)
print("Right")
display(right)

Left


,A,B,C,D,E
1,4,2,NaN,1,3
2,9,7,NaN,6,8


Right


,A,B,C,D,E
2,10,20,30,40,NaN
3,60,70,80,90,NaN
4,600,700,800,900,NaN


### Axis None, Outer Join

In [56]:
left, right = df.align(other, join="outer", axis=None)
print("Left")
display(left)
print("Right")
display(right)

Left


,A,B,C,D,E
1,4.0,2.0,NaN,1.0,3.0
2,9.0,7.0,NaN,6.0,8.0
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


Right


,A,B,C,D,E
1,NaN,NaN,NaN,NaN,NaN
2,10.0,20.0,30.0,40.0,NaN
3,60.0,70.0,80.0,90.0,NaN
4,600.0,700.0,800.0,900.0,NaN


### Join type: Inner Join

In [50]:
left, right = df.align(other, join="inner", axis=1)
print("Left")
display(left)
print("Right")
display(right)

Left


,D,B,A
1,1,2,4
2,6,7,9


Right


,D,B,A
2,40,20,10
3,90,70,60
4,900,700,600
